# IRA Tweets: Analysis on Italian Tweets



In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime
from zipfile import ZipFile

import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import networkx as nx 

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# 1. Data Reading and Exploration

### Reading the data

> In reading phase we read the input data (both zip files), we extract italian tweets and we preprocess it. Finally we store the result in two new csv files to an easy future access. We consider also another csv file with without italian tweets with region column equal to USA, since we have noticed that are difficult to interpret. 

First we iterate through "russian-troll-tweets.zip" to read through every .csv file:

In [2]:
# Read the first IRA dataset
data_folder = './data/'
zip_file = ZipFile(data_folder+'russian-troll-tweets.zip')
fs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}

In [3]:
fs['IRAhandle_tweets_1.csv'].head(5)

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


We append all datasets together in one DataFrame:

In [4]:
data=pd.DataFrame()
for key,val in fs.items():
    data=pd.concat([data,val])

In [5]:
data.describe()

,external_author_id,following,followers,updates,new_june_2018,retweet
count,2.973367e+06,2.973371e+06,2.973371e+06,2.973371e+06,2.973371e+06,2.973371e+06
mean,1.296128e+17,3.433524e+03,7.018913e+03,1.049756e+04,2.078735e-01,4.408955e-01
std,3.036341e+17,5.609881e+03,1.458463e+04,1.768729e+04,4.057859e-01,4.964945e-01
min,3.497640e+07,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.930748e+09,3.270000e+02,3.200000e+02,1.787000e+03,0.000000e+00,0.000000e+00
50%,2.581835e+09,1.499000e+03,1.274000e+03,4.333000e+03,0.000000e+00,0.000000e+00
75%,3.254274e+09,4.730000e+03,1.060000e+04,1.234100e+04,0.000000e+00,1.000000e+00
max,9.812510e+17,7.621000e+04,2.512760e+05,1.661130e+05,1.000000e+00,1.000000e+00


Here we preprocess the data, we notice that *content*, the main attribute of our analysis, has some null values. We prefer removing them. Finally we remove duplicate values of *content*. 

In [6]:
data.isnull().any()

external_author_id     True
author                False
content                True
region                 True
language              False
publish_date          False
harvested_date        False
following             False
followers             False
updates               False
post_type              True
account_type           True
new_june_2018         False
retweet               False
account_category      False
dtype: bool

In [7]:
data = data.dropna(subset= ['content'])
data.content.is_unique # Answer : False
data = data.drop_duplicates(subset = 'content')

We select only the italian tweets from the first 

In [8]:
# Italian tweets
italian_tweets = data[data.language == 'Italian']

# Italian tweets and not USA region
italian_tweets_notusa = italian_tweets[italian_tweets.region != 'United States']

In [9]:
italian_tweets.to_csv("italian_tweets_old.csv")
italian_tweets_notusa.to_csv("italian_tweets_old_notusa.csv")

We also import from a second dataset('ira_tweets_new.zip') and we repeat the previous analysis:

In [12]:
%%capture
# Read the second IRA dataset
zip_file = ZipFile(data_folder+'ira_tweets_new.zip')
interesting = ['rus_troll_tweet_text.csv',
               'rus_troll_tweet_stats.csv',
               'rus_troll_tweet_metadata.csv']
fs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename in interesting}

In [13]:
datanew=pd.DataFrame()
for key,val in fs.items():
    datanew=pd.concat([datanew,val], axis=1, sort=False)

In [14]:
datanew.isnull().any()

tweetid                 False
userid                  False
tweet_time              False
in_reply_to_tweetid      True
in_reply_to_userid       True
quoted_tweet_tweetid     True
is_retweet              False
retweet_userid           True
retweet_tweetid          True
quote_count             False
reply_count             False
like_count              False
retweet_count           False
hashtags                 True
urls                     True
user_mentions            True
poll_choices             True
tweetid                 False
tweet_language           True
tweet_text               True
tweetid                 False
follower_count          False
following_count         False
latitude                 True
longitude                True
tweet_client_name        True
dtype: bool

In [17]:
datanew = datanew.dropna(subset= ['tweet_text'])
datanew.tweet_text.is_unique # Answer : False
datanew = datanew.drop_duplicates(subset = 'tweet_text')

In [ ]:
data4 = pd.read_csv(zip_file.open('rus_troll_user.csv'))
italian_tweets = datanew[datanew.tweet_language == 'it']
italian_tweets = pd.merge(italian_tweets, data4, on ='userid')
italian_tweets.shape
italian_tweets.to_csv("italian_tweets.csv")
italian_tweets.head(20)

### Authors: following and followers

In [ ]:
italian_tweets = pd.read_csv('italian_tweets.csv')
italian_tweets_old = pd.read_csv('italian_tweets_old.csv')
italian_tweets.columns

In [ ]:
italian_tweets.head(5)

In [ ]:
italian_tweets_old.columns

In [ ]:
#italian_tweets['tweet_time'] =  pd.to_datetime(italian_tweets['tweet_time'], format='%Y/%m/%d %H:%M')
#italian_tweets_old['publish_date'] =  pd.to_datetime(italian_tweets_old['publish_date'], format='%m/%d/%Y %H:%M')

In [ ]:
italian_tweets['tweet_time'] =  pd.to_datetime(italian_tweets['tweet_time'], format='%Y/%m/%d %H:%M')
italian_tweets_old['publish_date'] =  pd.to_datetime(italian_tweets_old['publish_date'], format='%m/%d/%Y %H:%M')
italian_tweets=italian_tweets.rename(index=str, columns={"user_display_name":"author",
                                                         "follower_count_x": "followers",
                                                         "following_count_x": "following",
                                                         "tweet_time":"time"})
italian_tweets_old=italian_tweets_old.rename(index=str, columns={"publish_date":"time"})
italian_tweets=italian_tweets.rename(index=str, columns={"tweet_text":"content"})

In [ ]:
followers=pd.concat([italian_tweets[['author','followers','time']],italian_tweets_old[['author','followers','time']]])
followers=followers.groupby('author').agg({'followers': 'max'}).rename(columns={'followers':'max_followers'})

following=pd.concat([italian_tweets[['author','following','time']],italian_tweets_old[['author','following','time']]])
following=following.groupby('author').agg({'following': 'max'}).rename(columns={'following':'max_following'})

In [ ]:
avgfollowers=pd.concat([italian_tweets[['author','followers','time']],italian_tweets_old[['author','followers','time']]])
avgfollowers=avgfollowers.groupby('author').agg({'followers': 'mean'}).rename(columns={'followers':'mean_followers'})

avgfollowing=pd.concat([italian_tweets[['author','following','time']],italian_tweets_old[['author','following','time']]])
avgfollowing=avgfollowing.groupby('author').agg({'following': 'mean'}).rename(columns={'following':'mean_following'})

In [ ]:
fig=plt.figure(figsize=(20,13))
fig.suptitle('Visualization of the distribution of following/followers', fontsize=14, fontweight='bold')

ax = fig.add_subplot(221)
ax.hist(following.max_following,bins=100,log=True)
ax.set_xlabel('Number of Following(Max)')
ax.set_ylabel('Counts')
ax.set_title(r'Max-Following Number Distribution for Each Author')
plt.locator_params(axis='x',nbins=6)

ax2 = fig.add_subplot(222)
ax2.hist(followers.max_followers,bins=100,log=True)
ax2.set_xlabel('Number of Followers(Max)')
ax2.set_ylabel('Counts')
ax2.set_title(r'Max-Followers Number Distribution for Each Author')
plt.locator_params(axis='x',nbins=6)

ax3 = fig.add_subplot(223)
ax3.hist(avgfollowing.mean_following,bins=100,log=True)
ax3.set_xlabel('Number of Mean-Following(Mean)')
ax3.set_ylabel('Counts')
ax3.set_title(r'Mean-Following Number Distribution for Each Author')
plt.locator_params(axis='x',nbins=6)

ax4 = fig.add_subplot(224)
ax4.hist(avgfollowers.mean_followers,bins=100,log=True)
ax4.set_xlabel('Number of Mean-Followers(Mean)')
ax4.set_ylabel('Counts')
ax4.set_title(r'Mean-Followers Number Distribution for Each Author')
plt.locator_params(axis='x',nbins=6)

plt.show()

In [ ]:
counters=italian_tweets.groupby('author').count()
less_following = following.loc[following['max_following'] < 50000]
less_followers = followers.loc[followers['max_followers'] < 50000]

In [ ]:
fig=plt.figure(figsize=(10,4))
fig.suptitle('Visualization of the distribution of following/followers(Number<50000)', fontsize=14, fontweight='bold')
ax = fig.add_subplot(121)

ax.hist(less_following.max_following,bins=100, log=True)
ax.set_xlabel('Number of following')
ax.set_ylabel('Counts')
ax.set_title(r'Following')
plt.locator_params(axis='x',nbins=4)

ax2 = fig.add_subplot(122)
ax2.hist(less_followers.max_followers,bins=100,log=True)
ax2.set_xlabel('Number of followers')
ax2.set_ylabel('Counts')
ax2.set_title(r'Followers')

plt.show()

### Authors: activity

In [ ]:
italian_tweets=italian_tweets.rename(index=str, columns={"tweet_text":"content"})
authors=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
authors=authors.drop_duplicates(subset='content')
authors.head()

Here we count the total number of all contents:

In [ ]:
authors.content.count()

We find out that every content is unique:

In [ ]:
len(authors.content.unique())

In [ ]:
authors.dtypes

In [ ]:
authors['time']=authors['time'].apply(lambda x: x.date())#Only keep the date
#Count total number of tweets the author sent out in one day:
authors=authors.groupby(['author','time']).agg('count').rename(columns={'content':'tot'})
#Take the average number of tweets one author sent out per day:
authors=authors.groupby(by='author',as_index=True).agg({'tot':'mean'}).reset_index().rename(columns={'tot':'avg'})

In [ ]:
authors

In [ ]:
# fig=plt.figure(figsize=(10,4))

plt.hist(authors.avg, log=True)
plt.xlabel('Average # of Twitter Sent/Day')
plt.ylabel('Counts')
plt.title(r'Distribution of Twitters(Avg) Sent Every Day')
plt.show()

In [ ]:
tweets=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
tweets.describe()

In [ ]:
prolific=authors.loc[authors.avg>20]
tweets_prolific=tweets.copy().merge(prolific,how='right',on='author')
tweets_prolific=tweets_prolific[tweets_prolific['time'].apply(lambda x:x>datetime.strptime('01-01-2017', '%d-%m-%Y'))]
tweets_prolific.head()
#tweets.describe()

### Italian politicians

In [ ]:
tweets=tweets[tweets['time'].apply(lambda x:x>datetime.strptime('01-01-2017', '%d-%m-%Y'))]

In [ ]:
pd.set_option('display.max_colwidth', -1)
renzi=tweets_prolific[tweets_prolific['content'].apply(lambda x: 'renzi' in x.lower())]
renzi.content.count()

In [ ]:
pd.set_option('display.max_colwidth', -1)
renzi=tweets[tweets['content'].apply(lambda x: 'renzi' in x.lower())]
renzi.content.count()

In [ ]:
#renzi

In [ ]:
salvini=tweets_prolific[tweets_prolific['content'].apply(lambda x: 'salvini' in x.lower())]
salvini.content.count()

In [ ]:
salvini=tweets[tweets['content'].apply(lambda x: 'salvini' in x.lower())]
#salvini=tweets.where(lambda x: 'salvini' in x.lower())
salvini.content.count()

In [ ]:
grillo=tweets[tweets['content'].apply(lambda x: 'grillo' in x.lower())]
grillo.content.count()

In [ ]:
rep=tweets[tweets['content'].apply(lambda x: 'repubblica' in x.lower())]
rep.content.count()

### Geographical analysis 

In [ ]:
munic = pd.read_excel('comuni.xls')
munic=munic.rename(columns={'Denominazione in italiano':'comune','Denominazione regione':'regione'})[['comune','regione']]
munic.regione=munic.regione.apply(lambda x: x.lower())
munic['count']=0
munic=munic.set_index('comune')
munic.head()

In [ ]:
def wordInString(word, string_value):
    return True if re.search(r'\b' + word + r'\b', string_value) else False

In [ ]:
def count_c(name):
    tweet_comune=tweets[tweets['content'].apply(lambda x: wordInString(name, x))]
    total=tweet_comune.content.count()
    munic.at[name, 'count']=total

In [ ]:
def count_r(name):
    tweet_reg=tweets[tweets['content'].apply(lambda x: wordInString(name.lower(), x.lower()))]
    total=tweet_reg.content.count()
    reg.at[name, 'tot']=reg.loc[name,'tot']+total

In [ ]:
list_comuni=munic.index.tolist()
for m in list_comuni:
    count_c(m)

In [ ]:
reg=munic.groupby(by='regione')['count'].agg('sum').reset_index().rename(columns={'count':'tot'}).set_index('regione')

In [ ]:
list_regioni=reg.index.tolist()
for m in list_regioni:
    count_r(m)

In [ ]:
reg

### Time analysis

In [ ]:
times=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
#times.loc[times['time'].apply(lambda x:x==datetime.strptime('26-03-2017', '%d-%m-%Y'))]
times=times.drop_duplicates(subset='content')

In [ ]:
start_date_tweet = times['time'].min()
end_date_tweet = times['time'].max()
print(start_date_tweet, end_date_tweet)

In [ ]:
times.dtypes

In [ ]:
#times=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
times['time']=times['time'].apply(lambda x: x.date())#authors.head()
times['time'] = pd.to_datetime(times['time'])

In [ ]:
times=times.groupby(by='time',as_index=True).agg({'content':'count'}).reset_index().rename(columns={'content':'tot'})
times.sort_values(by='time', ascending = True, inplace = True)
times=times.set_index('time')

In [ ]:
times.dtypes

In [ ]:
times.head()

In [ ]:
plt.style.use('seaborn-darkgrid')
times['tot'].plot(linestyle = "-", figsize = (12,8), rot = 45, color = 'k',
                               linewidth = 1)
plt.title('Tweet counts by date', fontsize = 15)
plt.xlabel('Date', fontsize = 13)
plt.ylabel('Tweet Count', fontsize = 13)

plt.show()

In [ ]:
peak=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
peak=peak.drop_duplicates(subset='content')
peak=peak[peak['time'].apply(lambda x:
                         ((x<datetime.strptime('01-02-2017', '%d-%m-%Y') or 
                          (x>datetime.strptime('31-05-2017', '%d-%m-%Y'))) & 
                          (x>datetime.strptime('01-03-2014', '%d-%m-%Y')) & 
                          (x<datetime.strptime('15-11-2017', '%d-%m-%Y')))
                        )]
peak['time']=peak['time'].apply(lambda x: x.date())#authors.head()
peak['time'] = pd.to_datetime(peak['time'])
peak=peak.groupby(by='time',as_index=True).agg({'content':'count'}).reset_index().rename(columns={'content':'tot'})
peak.sort_values(by='time', ascending = True, inplace = True)
peak=peak.set_index('time')

dates_list = ['2016-12-01','2017-10-05']
# 08-2017
# March-May 2017 terror attacks in Europe: London, Stockholm, Paris, St Petersburg, Manchester
# 12-2016 Italian referendum
# 08-2015 Migratory crisis in Europe
# 05-2014 European elections
important_dates = pd.Series(pd.to_datetime(dates_list))

summer = ['2017-07-15','2016-08-15','2015-08-10','2014-08-10']
# 08-2017
# March-May 2017 terror attacks in Europe: London, Stockholm, Paris, St Petersburg, Manchester
# 12-2016 Italian referendum
# 08-2015 Migratory crisis in Europe
# 05-2014 European elections
summer_dates = pd.Series(pd.to_datetime(summer))

# add columns to identify important events, and mark a 0 or 1.
peak['Summer Events'] = False
peak.loc[summer_dates, 'Summer Events'] = True
peak['vals'] = 0
peak.loc[summer_dates, 'vals'] = 1

peak['Important Events'] = False
peak.loc[important_dates, 'Important Events'] = True
peak['values'] = 0
peak.loc[important_dates, 'values'] = 1

plt.style.use('seaborn-darkgrid')
peak['tot'].plot(linestyle = "-", figsize = (22,8), rot = 45, color = 'k',
                               linewidth = 1)
plt.title('Tweet counts by date', fontsize = 15)
plt.xlabel('Date', fontsize = 13)
plt.ylabel('Tweet Count', fontsize = 13)

plt.plot(peak[peak['Important Events'] == True].index.values,
         peak.loc[peak['Important Events'] == True, 'values'],
         marker = 'o', 
         color = 'r',
         linestyle = 'none',
        label = 'Important Dates in politics')

plt.plot(peak[peak['Summer Events'] == True].index.values,
         peak.loc[peak['Summer Events'] == True, 'vals'],
         marker = 'o', 
         color = 'b',
         linestyle = 'none',
        label = 'Summer migratory crisis')

for m in peak[peak['Summer Events'] == True].index.values:
    plt.axvline(x=m)
    
for m in peak[peak['Important Events'] == True].index.values:
    plt.axvline(x=m, color='r')

plt.legend(loc = 'best')

plt.show()

# 2. Data Mining

In [ ]:
italian_tweets = pd.read_csv('italian_tweets.csv')
italian_tweets_old = pd.read_csv('italian_tweets_old_notusa.csv')

In [ ]:
italian_tweets['tweet_time'] =  pd.to_datetime(italian_tweets['tweet_time'], format='%Y/%m/%d %H:%M')
italian_tweets_old['publish_date'] =  pd.to_datetime(italian_tweets_old['publish_date'], format='%m/%d/%Y %H:%M')
italian_tweets=italian_tweets.rename(index=str, columns={"user_display_name":"author",
                                                         "follower_count_x": "followers",
                                                         "following_count_x": "following",
                                                         "tweet_time":"time"})
italian_tweets_old=italian_tweets_old.rename(index=str, columns={"publish_date":"time"})
italian_tweets=italian_tweets.rename(index=str, columns={"tweet_text":"content"})
ita=pd.concat([italian_tweets[['content','author','time']],italian_tweets_old[['content','author','time']]])
ita=ita.drop_duplicates(subset='content')
regex = re.compile('[^a-zA-Z ]')
def returntest(text):
    text=text.lower()
    text=regex.sub('',text) 
    text=re.sub(r'(\s)http\w+', r' ', text)
    return text
new_text = ita.content.apply(returntest)
print(new_text.head(3))
new_text.to_csv("tweets_ita.csv")

### Association Rules

In [ ]:
def draw_graph(rules, rules_to_show,strs):
    G1 = nx.DiGraph()
    color_map=[]
    N = 50
    colors = np.random.rand(N)       
   
    for i in range (rules_to_show):      
        G1.add_nodes_from(["R"+str(i)])
    
        for a in rules.iloc[i]['antecedents']:
            G1.add_nodes_from([a])
            G1.add_edge(a, "R"+str(i), color=colors[i] , weight = 2)
       
        for c in rules.iloc[i]['consequents']:
            G1.add_nodes_from([c])
            G1.add_edge("R"+str(i), c, color=colors[i],  weight=2)
    
    for node in G1:
        found_a_string = False
        
        for item in strs: 
            if node==item:
                found_a_string = True
        if found_a_string:
            color_map.append('yellow')
        else:
            color_map.append('red')       
 
 
    edges = G1.edges()
    colors = [G1[u][v]['color'] for u,v in edges]
    weights = [G1[u][v]['weight'] for u,v in edges]
 
    pos = nx.spring_layout(G1, k=16, scale=1)
    nx.draw(G1, pos, edges=edges, node_color = color_map, edge_color=colors, width=weights, font_size=16, with_labels=False)            
   
    for p in pos:  # raise text positions
        pos[p][1] += 0.07
    
    nx.draw_networkx_labels(G1, pos)
    plt.show()

In [ ]:
italian_tweets_text = pd.read_csv('tweets_ita.csv',header=None)
print(italian_tweets_text.shape)
stopwords = pd.read_json('stopwords-it.json')

In [ ]:
stopwords = stopwords[0].tolist()

In [ ]:
list_final = []
for text in italian_tweets_text[1]:
    list_final.append([word.lower() for word in text.split() 
                       if (word.lower() not in stopwords and
                           (len(word)>3 or word in ['pd','ms']))])
    
list_final

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(list_final).transform(list_final)
df = pd.DataFrame(te_ary,columns=te.columns_)

In [ ]:
frequent_items = apriori(df, min_support=0.001, use_colnames=True, max_len = 2)

In [ ]:
couple = frequent_items [frequent_items['itemsets'].apply(lambda x: len(x)==2)]
couple = couple.sort_values(by=['support'],ascending= False)
couple.head()

In [ ]:
table = association_rules(frequent_items, metric="confidence", min_threshold=0.2)
table = table.sort_values(by=['confidence'],ascending= False)

In [ ]:
pd = table[(table['antecedents'].apply(lambda x: 'pd' in str(x))) | (table['consequents'].apply(lambda x: 'pd' in str(x)))]
strs=['R0', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9','R10', 'R11', 'R12', 'R13']
draw_graph(pd, 14, strs)  

In [ ]:
ms = table[(table['antecedents'].apply(lambda x: 'ms' in str(x))) | (table['consequents'].apply(lambda x: 'ms' in str(x)))]
strs=['R0', 'R1', 'R2', 'R3']
draw_graph(ms, 4, strs)

In [ ]:
text = " ".join([word for tweet in list_final for word in tweet])
wordcloud = WordCloud(background_color="white").generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
papers=['Corriere della Sera','repubblica','repubblicait','SoleORE','QuotidianoNazionale',
        'Gazzettait','laStampa','ilMessaggeroit','quotidianonazionale','ilpost',
        'corriere','ilgiornale','Avvenire','Tuttosport','libero',
        'FattoQuotidiano','Mattinodinapoli','huffpost','linkiesta',
        'adnkronos','agenziaansa','video','giornalettismo','ilpost',
        'lettera','huffpostitalia','agenziaitalia','skytg','skysport','internazionale',
        'perch','foto','repubblicatv','wireditalia','news','diretta','leggo','radiorai']
papers=[m.lower() for m in papers]
text = " ".join([word for tweet in list_final for word in tweet if word not in papers])
wordcloud = WordCloud(background_color="white").generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()